<a href="https://colab.research.google.com/github/samtam0714/ML/blob/master/EnsModel_regipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#Load Dataset
data=pd.read_csv('./mtcars.csv')
data.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [ ]:
#Remove Model from Dataset
data2=data.drop('model',axis=1)
data2.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [ ]:
#Show Key Statistics
data2.describe()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.0000
mean,20.090625,6.187500,230.721875,146.687500,3.596563,3.217250,17.848750,0.437500,0.406250,3.687500,2.8125
std,6.026948,1.785922,123.938694,68.562868,0.534679,0.978457,1.786943,0.504016,0.498991,0.737804,1.6152
min,10.400000,4.000000,71.100000,52.000000,2.760000,1.513000,14.500000,0.000000,0.000000,3.000000,1.0000
25%,15.425000,4.000000,120.825000,96.500000,3.080000,2.581250,16.892500,0.000000,0.000000,3.000000,2.0000
50%,19.200000,6.000000,196.300000,123.000000,3.695000,3.325000,17.710000,0.000000,0.000000,4.000000,2.0000
75%,22.800000,8.000000,326.000000,180.000000,3.920000,3.610000,18.900000,1.000000,1.000000,4.000000,4.0000
max,33.900000,8.000000,472.000000,335.000000,4.930000,5.424000,22.900000,1.000000,1.000000,5.000000,8.0000


In [ ]:
#Define x and y variable
x = data2.drop('mpg',axis=1).values
y = data2['mpg'].values

In [ ]:
#Train dataset using 20% test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

#Scale the Data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train2 = sc.fit_transform(x_train)
x_test2 = sc.fit_transform(x_test)

x_2 = sc.fit_transform(x)

In [ ]:
#Prepare Models
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from xgboost import XGBRegressor

alpha=0.5
models = [LinearRegression(),Lasso(alpha=alpha,random_state=100),Ridge(alpha=alpha,random_state=100),
          ElasticNet(alpha=alpha,random_state=100),RandomForestRegressor(random_state=100),
          BaggingRegressor(random_state=100),AdaBoostRegressor(random_state=100),
          GradientBoostingRegressor(random_state=100),XGBRegressor(random_state=100)]

In [ ]:
#Create Model Comparison
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate

#Scoring Parameters
scoring = {'MAE':'neg_mean_absolute_error',
           'MSE':'neg_mean_squared_error'}

MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

row_index = 0
for alg in models:

    predicted = alg.fit(x_train2, y_train).predict(x_test2)
    rmse_test = np.sqrt(metrics.mean_squared_error(y_test, predicted))

    rkfcv = alg.fit(x_train2, y_train)
    rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=100)
    rKFcv = cross_validate(rkfcv, x_2, y, scoring=scoring,
                         cv=rkf)

    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'Algorithm Name'] = MLA_name
    MLA_compare.loc[row_index, 'MAE'] = round(-rKFcv['test_MAE'].mean(),2)
    MLA_compare.loc[row_index, 'RMSE'] = rmse_test.round(2)
    MLA_compare.loc[row_index, 'rRMSE_10cv'] = round(np.sqrt(-rKFcv['test_MSE'].mean()),2)

    row_index+=1

MLA_compare.sort_values(by = ['rRMSE_10cv'], ascending = True, inplace = True)
MLA_compare

,Algorithm Name,MAE,RMSE,rRMSE_10cv
7,GradientBoostingRegressor,1.88,3.13,2.59
4,RandomForestRegressor,2.07,2.27,2.64
8,XGBRegressor,2.09,3.11,2.65
5,BaggingRegressor,2.10,2.58,2.72
6,AdaBoostRegressor,2.10,2.13,2.83
3,ElasticNet,2.33,1.64,2.87
1,Lasso,2.41,1.46,2.99
2,Ridge,2.71,2.88,3.30
0,LinearRegression,2.94,3.74,3.67


In [ ]:
#Create Stacking Model
from mlxtend.regressor import StackingCVRegressor

#Identify Models
lm = ElasticNet(alpha=alpha,random_state=100)
mod1 = RandomForestRegressor(random_state=100)
mod2 = XGBRegressor(random_state=100)
mod3 = BaggingRegressor(random_state=100)

#Create Stacking Classifier
np.random.seed(100)
stackmod=StackingCVRegressor(regressors=[mod1,mod2,mod3],
                             meta_regressor=lm)

scoring = {'MAE':'neg_mean_absolute_error',
           'MSE':'neg_mean_squared_error'}

for clf in (mod1,mod2,mod3,stackmod):
    rkfcv= clf.fit(x_train2,y_train)
    ens_rkf1 = RepeatedKFold(n_splits=10, n_repeats=5, random_state=100)
    rKFcv = cross_validate(rkfcv, x_2, y, scoring=scoring, cv=ens_rkf1)
    print(clf.__class__.__name__,round(np.sqrt(-rKFcv['test_MSE'].mean()),2))

RandomForestRegressor 2.64
XGBRegressor 2.65
BaggingRegressor 2.72
StackingCVRegressor 2.55
